In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Final Project: NBA Quick Stats

This is a test.

In [7]:
url = 'https://www.basketball-reference.com/leagues/NBA_2021.html'
tables = pd.read_html(url)
tables[4].sort_values('PTS', ascending=False)

,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1.0,Milwaukee Bucks*,72,240.7,44.7,91.8,0.487,14.4,37.1,0.389,...,0.760,10.3,37.8,48.1,25.5,8.1,4.6,13.8,17.3,120.1
1,2.0,Brooklyn Nets*,72,241.7,43.1,87.3,0.494,14.2,36.1,0.392,...,0.804,8.9,35.5,44.4,26.8,6.7,5.3,13.5,19.0,118.6
2,3.0,Washington Wizards*,72,241.7,43.2,90.9,0.475,10.2,29.0,0.351,...,0.769,9.7,35.5,45.2,25.5,7.3,4.1,14.4,21.6,116.6
3,4.0,Utah Jazz*,72,241.0,41.3,88.1,0.468,16.7,43.0,0.389,...,0.799,10.6,37.6,48.3,23.7,6.6,5.2,14.2,18.5,116.4
4,5.0,Portland Trail Blazers*,72,240.3,41.3,91.1,0.453,15.7,40.8,0.385,...,0.823,10.6,33.9,44.5,21.3,6.9,5.0,11.1,18.9,116.1
5,6.0,Phoenix Suns*,72,242.8,43.3,88.3,0.490,13.1,34.6,0.378,...,0.834,8.8,34.2,42.9,26.9,7.2,4.3,12.5,19.1,115.3
6,7.0,Indiana Pacers,72,242.4,43.3,91.2,0.474,12.3,34.0,0.364,...,0.792,9.0,33.7,42.7,27.4,8.5,6.4,13.5,20.2,115.3
7,8.0,Denver Nuggets*,72,242.8,43.3,89.2,0.485,12.9,34.2,0.377,...,0.803,10.5,33.9,44.4,26.8,8.1,4.5,13.5,19.1,115.1
8,9.0,New Orleans Pelicans,72,242.1,42.5,89.1,0.477,10.6,30.4,0.348,...,0.729,11.7,35.7,47.4,26.0,7.6,4.4,14.6,18.0,114.6
9,10.0,Los Angeles Clippers*,72,240.0,41.8,86.7,0.482,14.3,34.7,0.411,...,0.839,9.4,34.7,44.2,24.4,7.1,4.1,13.2,19.2,114.0


Questions

(1) How have the basic stats (ie. 3-point FGs attempted, winning percentage, etc.) of NBA teams changed over the past decade?
Ryan
(2) Playoff team data vs Non-playoff team data over past decade
Chris
(3) What variable has the highest correlation with winning percentage among teams in the top 25% over the past decade? Is there a significant difference with this variable between the top 25% and bottom 25%?
Jovanni